score = 0.4504 

Here, we combine the results of the networks in order to improve the prediction capability

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import tensorflow as tf
import numpy as np
cwd = os.getcwd()
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [4]:
from keras.models import load_model
model_mais = load_model('/content/drive/MyDrive/second_custom_network_Mais_Dec28_13-20-45.h5',custom_objects={'meanIoU':meanIoU})
model_haricot = load_model('/content/drive/MyDrive/second_custom_network_Dec28_10-28-05_Haricot.h5',custom_objects={'meanIoU':meanIoU})

In [5]:
img_h = 384
img_w = 512
from PIL import Image

class CustomDataSol(tf.keras.utils.Sequence):

  def __init__(self, dataset_dir, img_generator=None,
               preprocessing_function=None, out_shape=[512, 384],projects=['Weedelec','Pead','Roseau','Bipbip'],plants=['Haricot','Mais']):

    self.subset_filenames = []
    self.subset_direc = []
    self.name_project = []
    self.name_plant = []
    self.projects = []
    self.plants = []
    self.dataset_dir = dataset_dir
    self.img_generator = img_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape

    for project in projects :
      for plant in plants :
        path=os.path.join(dataset_dir,project,plant)
        list = os.listdir(os.path.join(path,'Images'))
        number_files = len(list)
        for ii in range(0,number_files):
            self.projects.append(project)
            self.plants.append(plant)
            self.subset_filenames.append(os.path.splitext(list[ii])[0])
            self.subset_direc.append(path)
            self.name_project.append(project)
            self.name_plant.append(plant)
    
  def __len__(self):
    return len(self.subset_filenames)

  def __getitem__(self, index):
    # Read Image
    curr_filename = self.subset_filenames[index]
    curr_dir = self.subset_direc[index]
    curr_project = self.projects[index]
    if curr_project == 'Roseau' :
      img = Image.open(os.path.join(curr_dir, 'Images', curr_filename + '.png'))
    else :
      img = Image.open(os.path.join(curr_dir, 'Images', curr_filename + '.jpg'))

    # Resize image 
    img = img.resize(self.out_shape)
    img_arr = np.array(img)
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr

################################################################################
 # Encode array (start kit)

def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)


################################################################################

path_test = '/content/drive/My Drive/Development_Dataset/Test_Dev/'
test_set=CustomDataSol(path_test, 
                        img_generator=None,
                        projects=['Weedelec','Pead','Roseau','Bipbip'],
                        plants=['Mais','Haricot'])
test_dataset = tf.data.Dataset.from_generator(lambda: test_set,
                                               output_types= tf.float32,
                                               output_shapes=[img_h, img_w, 3])

test_dataset = test_dataset.batch(1)
iterator = iter(test_dataset)
test_set.__len__()
    
submission_dict = {}
SIZE=[[1536,2048],[2464,3280],[819,1228],[3456,5184]]
for index in range(0,test_set.__len__()):#
      image = next(iterator)
      if test_set.projects[index]=='Bipbip':
        size_im=SIZE[0]
      elif test_set.projects[index]=='Pead':
        size_im=SIZE[1]
      elif test_set.projects[index]=='Roseau':
        size_im=SIZE[2]
      else:
        size_im=SIZE[3]
      img_name = os.path.splitext(test_set.subset_filenames[index])[0]
      submission_dict[img_name] = {}
      if test_set.plants[index] == 'Mais':
        model = model_mais
      else:
        model = model_haricot
      mask_arr =  model.predict(image)
      mask_arr = tf.image.resize(mask_arr, size_im, method='nearest')
      #mask_arr.size
      predicted_class = tf.argmax(mask_arr, -1)
      predicted_class = predicted_class[0, ...]
      prediction_img = np.zeros([size_im[0], size_im[1]])
      #prediction_img2 = np.zeros([size_im[0], size_im[1], 3])
  
      prediction_img[np.where(predicted_class == 0)] = 0
      #prediction_img2[np.where(predicted_class == 0)] = [0, 0, 0]
      for i in range(1, 3):
        prediction_img[np.where(predicted_class == i)] = i
        #prediction_img2[np.where(predicted_class == i)] = np.array(colors[i-1])[:3] * 255
      #fig, ax = plt.subplots(1, 2, figsize=(10, 10))
      #fig.show() 
      #print(np.uint8(prediction_img2).size)   
      #ax[0].imshow(np.uint8(prediction_img2))
      #ax[1].imshow(np.uint8(image[0]))
      #fig.canvas.draw()
      #time.sleep(1)

      submission_dict[img_name]['shape'] = size_im
      submission_dict[img_name]['team'] = test_set.projects[index]
      submission_dict[img_name]['crop'] = test_set.plants[index]
      submission_dict[img_name]['segmentation'] = {}
    
    # crop
      rle_encoded_crop = rle_encode(prediction_img == 1)
    # weed
      rle_encoded_weed = rle_encode(prediction_img == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

import json
with open('/content/drive/My Drive/submission.json', 'w') as file:
      json.dump(submission_dict, file)

120

In [6]:
from google.colab import files
files.download("/content/drive/MyDrive/submission.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>